In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
x_train_7 = x_train[y_train < 7]
y_train_7 = y_train[y_train < 7]
x_test_7 = x_test[y_test < 7]
y_test_7 = y_test[y_test < 7]
input_shape = (28, 28, 1)

In [4]:
def format_data(train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)
    return x_train, x_test, y_train, y_test

In [5]:
x_train_7, x_test_7, y_train_7, y_test_7 = format_data((x_train_7, y_train_7), (x_test_7, y_test_7), 7)

In [6]:
Feature_layers = [
    Conv2D(32, (3, 3), padding='valid', input_shape=input_shape),
    Activation('relu'),
    Conv2D(32, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
]

In [7]:
Classification_layers_7 = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(7),
    Activation('softmax')
]

In [8]:
model_7 = Sequential(Feature_layers + Classification_layers_7)

In [9]:
model_7.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [10]:
model_7.compile(loss='categorical_crossentropy',
                optimizer='adadelta',
                metrics=['accuracy'])

In [11]:
model_7.fit(x_train_7, y_train_7,
            batch_size=32,
            epochs=5,
            verbose=1,
            validation_data=(x_test_7, y_test_7))

Epoch 1/5


1311/1311 [==============================] - 37s 26ms/step - loss: 1.9124 - accuracy: 0.1991 - val_loss: 1.8424 - val_accuracy: 0.4208
Epoch 2/5
1311/1311 [==============================] - 33s 25ms/step - loss: 1.7920 - accuracy: 0.4014 - val_loss: 1.6844 - val_accuracy: 0.7728
Epoch 3/5
1311/1311 [==============================] - 33s 25ms/step - loss: 1.6161 - accuracy: 0.5744 - val_loss: 1.4436 - val_accuracy: 0.8266
Epoch 4/5
1311/1311 [==============================] - 33s 25ms/step - loss: 1.3750 - accuracy: 0.6768 - val_loss: 1.1415 - val_accuracy: 0.8516
Epoch 5/5
1311/1311 [==============================] - 33s 25ms/step - loss: 1.1185 - accuracy: 0.7334 - val_loss: 0.8610 - val_accuracy: 0.8668


In [12]:
score_7 = model_7.evaluate(x_test_7, y_test_7, verbose=0)
print(f"Test accuracy on digits 0-6: {score_7[1] * 100:.2f}%")

Test accuracy on digits 0-6: 86.68%


#  Freeze the Feature_layers 

In [13]:
for layer in Feature_layers:
    layer.trainable = False

In [14]:
x_train_7to9 = x_train[y_train >= 7]
y_train_7to9 = y_train[y_train >= 7] - 7  # To make labels 0, 1, 2
x_test_7to9 = x_test[y_test >= 7]
y_test_7to9 = y_test[y_test >= 7] - 7  # To make labels 0, 1, 2

In [15]:
x_train_7to9, x_test_7to9, y_train_7to9, y_test_7to9 = format_data((x_train_7to9, y_train_7to9), (x_test_7to9, y_test_7to9), 3)

In [16]:
Classification_layers_3 = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(3),
    Activation('softmax')
]

In [17]:
model_7to9 = Sequential(Feature_layers + Classification_layers_3)

In [18]:
model_7to9.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                      

# Here we can see that the number of trainable parameters is reduced because we froze the earlier layers.

In [19]:
model_7to9.compile(loss='categorical_crossentropy',
                   optimizer='adadelta',
                   metrics=['accuracy'])

In [20]:
model_7to9.fit(x_train_7to9, y_train_7to9,
               batch_size=32,
               epochs=5,
               verbose=1,
               validation_data=(x_test_7to9, y_test_7to9))

Epoch 1/5
565/565 [==============================] - 13s 19ms/step - loss: 1.0836 - accuracy: 0.3991 - val_loss: 0.9627 - val_accuracy: 0.6852
Epoch 2/5
565/565 [==============================] - 11s 19ms/step - loss: 0.9522 - accuracy: 0.5857 - val_loss: 0.8463 - val_accuracy: 0.8579
Epoch 3/5
565/565 [==============================] - 10s 18ms/step - loss: 0.8560 - accuracy: 0.7007 - val_loss: 0.7542 - val_accuracy: 0.8838
Epoch 4/5
565/565 [==============================] - 11s 19ms/step - loss: 0.7761 - accuracy: 0.7586 - val_loss: 0.6794 - val_accuracy: 0.8960
Epoch 5/5
565/565 [==============================] - 11s 19ms/step - loss: 0.7084 - accuracy: 0.7965 - val_loss: 0.6172 - val_accuracy: 0.9000


In [21]:
score_7to9 = model_7to9.evaluate(x_test_7to9, y_test_7to9, verbose=0)
print(f"Test accuracy on digits 7-9: {score_7to9[1] * 100:.2f}%")

Test accuracy on digits 7-9: 90.00%


# Now use the Pretrained model i.e Model_7

In [22]:
model_7.pop() ##drop softmax layer

In [23]:
model_7.pop()## drop Dense(7) beacuse it classifies 7 class

In [24]:
model_7.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [25]:
model_7.add(Dense(3))
model_7.add(Activation("softmax")) ## add dense layer with 3 classes

In [26]:
model_7.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [27]:
model_7.compile(loss='categorical_crossentropy',
                   optimizer='adadelta',
                   metrics=['accuracy'])

In [28]:
model_7.fit(x_train_7to9, y_train_7to9,
               batch_size=32,
               epochs=5,
               verbose=1,
               validation_data=(x_test_7to9, y_test_7to9))

Epoch 1/5
565/565 [==============================] - 12s 19ms/step - loss: 1.0966 - accuracy: 0.4085 - val_loss: 0.9208 - val_accuracy: 0.6891
Epoch 2/5
565/565 [==============================] - 10s 18ms/step - loss: 0.9530 - accuracy: 0.5414 - val_loss: 0.8005 - val_accuracy: 0.8310
Epoch 3/5
565/565 [==============================] - 10s 18ms/step - loss: 0.8444 - accuracy: 0.6452 - val_loss: 0.7034 - val_accuracy: 0.8625
Epoch 4/5
565/565 [==============================] - 10s 18ms/step - loss: 0.7482 - accuracy: 0.7258 - val_loss: 0.6233 - val_accuracy: 0.8831
Epoch 5/5
565/565 [==============================] - 10s 18ms/step - loss: 0.6746 - accuracy: 0.7722 - val_loss: 0.5577 - val_accuracy: 0.8937
